In [12]:
import pandas as pd
import numpy.random as npr
from sdv.tabular import CTGAN, GaussianCopula, CopulaGAN
from sdv.evaluation import evaluate

DATASET_PATH = "data.csv"
USER_COLUMNS = ["user_id", "gender", "occupation", "age"]
MOVIE_COLUMNS = ["movie_id", "movie_name", "tag0", "tag1"]
SEED = 2021


In [13]:
df = pd.read_csv(DATASET_PATH)
df

/Users/vldpro/Workspace/university/recsys/.venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3170: DtypeWarning: Columns (26) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,user_id,movie_id,rating,timestamp,movie_name,tag0,tag1,tag2,tag3,tag4,...,tag12,tag13,tag14,tag15,tag16,tag17,gender,age,occupation,zip
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,F,1,10,48067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,4211,3791,2,965319075,Footloose (1984),0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,M,45,5,77662
1000205,4211,3806,3,965319138,MacKenna's Gold (1969),0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,M,45,5,77662
1000206,4211,3840,4,965319197,Pumpkinhead (1988),0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,M,45,5,77662
1000207,4211,3766,2,965319138,Missing in Action (1984),0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,M,45,5,77662


In [ ]:
npr.seed(SEED)

In [19]:
user_df = df[USER_COLUMNS].drop_duplicates().drop(["user_id"], axis=1)
user_df

,gender,occupation,age
0,F,10,1
53,M,16,56
182,M,12,25
205,M,7,25
406,M,1,50
...,...,...,...
1000035,M,10,1
1000092,M,2,18
1000113,M,4,25
1000135,M,6,25


In [20]:
movie_df = df[MOVIE_COLUMNS].drop_duplicates().drop(["movie_id"], axis=1)
movie_df

,movie_name,tag0,tag1
0,One Flew Over the Cuckoo's Nest (1975),0.0,0.0
1,James and the Giant Peach (1996),1.0,0.0
2,My Fair Lady (1964),0.0,0.0
3,Erin Brockovich (2000),0.0,0.0
4,"Bug's Life, A (1998)",1.0,0.0
...,...,...,...
996122,"Baby, The (1973)",0.0,0.0
996169,Running Free (2000),0.0,0.0
996915,Open Season (1996),0.0,0.0
997414,Aiqing wansui (1994),0.0,0.0


In [21]:
def fit_evaluate(df, models, sample_size):
    result = pd.DataFrame()
    for model_name, model_factory in models.items():
        model = model_factory()
        model.fit(df)
        eval_result = evaluate(df, model.sample(sample_size), aggregate=False)
        eval_result["model_name"] = name
        result = pd.concat([result, eval_result])
    return result
        
    

In [ ]:
fit_evaluate(
    movie_df, 
    models={
        "GaussianCopula": GaussianCopula,
        "CTGAN": CTGAN,
        "CopulaGAN": CopulaGAN
    },
    sample_size=1000
)

/Users/vldpro/Workspace/university/recsys/.venv/lib/python3.7/site-packages/scipy/stats/_continuous_distns.py:547: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/Users/vldpro/Workspace/university/recsys/.venv/lib/python3.7/site-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/Users/vldpro/Workspace/university/recsys/.venv/lib/python3.7/site-packages/scipy/stats/_continuous_distns.py:4798: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
/Users/vldpro/Workspace/university/recsys/.venv/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:2407: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
